# Final Model Training

In [ ]:
import scipy
import numpy as np
import pandas as pd
import lightfm
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from helpers.training import construct_item_features
from lightfm import LightFM
from lightfm import data
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score, reciprocal_rank
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
np.random.seed(123)

In [ ]:
filtered = pd.read_parquet('data/filtered.parquet', engine='pyarrow')

In [ ]:
light_data = lightfm.data.Dataset()
metadata = list(set(filtered['style'].unique()).union(set(filtered['brewery'].unique())))
light_data.fit(users=filtered['user'].unique(),
               items=filtered['beer_id'].unique(),
               item_features=metadata)
user_mappings = light_data._user_id_mapping
item_mappings = light_data._item_id_mapping
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}

In [ ]:
interactions = light_data.build_interactions(filtered[['user', 'beer_id', 'weight_all']].values)

In [ ]:
with open('artifacts/best_params_with_item_features.pkl', 'rb') as f:
    item_feature_best_params= pickle.load(f)

In [ ]:
features = construct_item_features(light_data, filtered, item_feature_best_params['b'], item_feature_best_params['s'])
best_model_item_features = LightFM(**{k: v for k, v in item_feature_best_params.items() if k not in ['epochs', 'b', 's']}, random_state=123)
best_model_item_features.fit(interactions[0], 
               sample_weight=interactions[1],
               item_features=features,
               epochs=item_feature_best_params['epochs'], 
               verbose=True)

In [ ]:
# Upload final model to your database! Happy brewing :)